In [1]:
import torch
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertConfig, BertPreTrainedModel, BertForPreTraining, BertForMaskedLM
import torch.nn as nn
from tqdm import tqdm, tqdm_notebook
import os
RUBERT_PATH = 'D:\\models\\rubert_cased_L-12_H-768_A-12_pt'
modelpath = os.path.join(RUBERT_PATH,'pytorch_model.bin')

In [ ]:
#!pip install transformers #huggingface

In [2]:
os.path.isfile(os.path.join(RUBERT_PATH,'pytorch_model.bin'))

True

In [14]:
# tokenizer = BertTokenizer.from_pretrained(os.path.join(RUBERT_PATH,'vocab.txt'))
tokenizer = BertTokenizer.from_pretrained(RUBERT_PATH, do_lower_case=False)
config = BertConfig.from_json_file(os.path.join(RUBERT_PATH,'bert_config.json'))
model = BertForPreTraining.from_pretrained(modelpath, config=config)
# model.eval()
# model = BertForMaskedLM.from_pretrained(modelpath, config=config)
model.eval()

# model = BertForPreTraining.from_pretrained(RUBERT_PATH)


BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [15]:
text = u"Пассиона́рии — в пассионарной теории этногенеза люди, обладающие врождённой способностью абсорбировать из внешней среды энергии больше, чем это требуется только для личного и видового самосохранения, и выдавать эту энергию в виде целенаправленной работы по видоизменению окружающей их среды. Судят о повышенной пассионарности того или иного человека по характеристике его поведения и психики."
target = u"психики"
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)
# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = tokenized_text.index(target)
tokenized_text[masked_index] = '[MASK]'
print(masked_index)

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [1] * len(tokenized_text)
# this is for the dummy first sentence. 
segments_ids[0] = 0
segments_ids[1] = 0

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
print(tokens_tensor.shape)
print(tokens_tensor)
segments_tensors = torch.tensor([segments_ids])
# Load pre-trained model (weights)
# model = BertForMaskedLM.from_pretrained(modelpath)
# model.eval()

# Predict all tokens
predictions = model(tokens_tensor, token_type_ids=segments_tensors)
print(predictions[0].shape)
predicted_index = torch.argmax(predictions[0][0][masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
print("Original:", text)
print("Masked:", " ".join(tokenized_text))
print(predictions[0].shape)
print("Predicted token:", predicted_token)
print("Other options:")
# just curious about what the next few options look like.
for i in range(10):
#     predictions[0,masked_index,predicted_index] = -11100000
    predicted_index = torch.argmax(predictions[0][0][masked_index]).item()
    print(predicted_index)
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
    print(predicted_token)

['Пасс', '##иона', '##́ри', '##и', '—', 'в', 'пасс', '##иона', '##р', '##ной', 'теории', 'этноген', '##еза', 'люди', ',', 'обладающие', 'врожд', '##ённой', 'способностью', 'абсорб', '##ировать', 'из', 'внешней', 'среды', 'энергии', 'больше', ',', 'чем', 'это', 'требуется', 'только', 'для', 'личного', 'и', 'видов', '##ого', 'самос', '##охран', '##ения', ',', 'и', 'выдавать', 'эту', 'энергию', 'в', 'виде', 'целенаправленно', '##й', 'работы', 'по', 'видоизмен', '##ению', 'окружающей', 'их', 'среды', '.', 'Судя', '##т', 'о', 'повышенной', 'пасс', '##иона', '##р', '##ности', 'того', 'или', 'иного', 'человека', 'по', 'характеристике', 'его', 'поведения', 'и', 'психики', '.']
73
torch.Size([1, 75])
tensor([[ 56493,   9042, 114177,    852,    901,    845,  32038,   9042,    874,
           1700,  14422,  99221,  16182,  11894,    128,  62824,  47474,  11827,
          38843,  81836,   7996,   1703,  20895,  18624,  17754,   8980,    128,
           3622,   3998,  23145,   4564,   2748,  23332,

In [ ]:
from torch import load
di = load(modelpath)
# with open(modelpath,encoding='utf-8') as fh:
print(type(di))

In [ ]:
print(di)